In [5]:
from keras.models import Sequential, ask_to_proceed_with_overwrite
from keras.layers import Dense
from keras.layers.core import Activation, Dense
from keras.layers import SimpleRNN, LSTM,  GRU,SeparableConvolution2D, Conv2D
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Dense, GlobalAveragePooling2D
import glob
import scipy
import numpy as np
from PIL import Image
import random
import pandas as pd
import keras
from keras.applications import VGG16, VGG19
from keras.models import Model
from keras.layers.core import Activation, Dense
import tensorflow as tf

def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    #vgg_model = VGG16( weights='imagenet', include_top=False )
    vgg_model = VGG16(weights='imagenet', include_top=False)

    for layer in vgg_model.layers[:-3]:
        layer.trainable = False

    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(output_dim ,activation='softmax')(x)

# this is the model we will train
    model = Model(input=vgg_model.input, output=predictions)

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate

    return model


def load_model(weights_path, output_dim):

    model = getModel( output_dim ) 
    for k,layer in model.layers_by_depth.items()[1:]:
        layer[0].trainable = False
    
    model.load_weights(weights_path)
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'categorical_crossentropy'])   
    return model

In [6]:
def load_data(N,output_dim):
    X = []
    y = []
    
    
    filenames = sorted(glob.glob("../logo_dataset_6/*/*.jpg"))
    junk_filenames = glob.glob("../junk_processed/*/*.jpg")
    
    random.shuffle(junk_filenames)
    
    junk_filenames = junk_filenames[:len(filenames)]
    filenames += junk_filenames
    
    random.shuffle(filenames)
    X_mean = 0.0
    for filename in filenames[:N]:
        label = np.zeros(output_dim,)
        idx = labels_dict.get(filename.split("/")[2], 6)
        label[idx] = 1.0
        img = np.asarray(Image.open(filename).resize((224, 224), Image.ANTIALIAS))/255.
        if img.shape == (224,224,3):
            X_mean += img.mean()
            y.append(label)
            X.append(img)

        
    limit = len(X)/10*8
    print(len(X))
    X_mean = X_mean/float(len(X))
    X_train = np.array(X[:limit])
    X_val = np.array(X[limit:])
    y_train = np.array(y[:limit])
    y_val = np.array(y[limit:])   
    X_train -= X_mean
    X_val -= X_mean
    print "completed formatting!"
    return X_train, y_train, X_val, y_val

In [7]:
labels_dict = {"Cocacola":0, "LOreal":1,"LouisVuitton":2,"Nike":3,"Pepsi":4,"Samsung":5}

In [8]:
X_train, y_train, X_val, y_val = load_data(100000,7 )

2820
completed formatting!


In [ ]:
output_dim = 7
epochs = 100
model = getModel( output_dim ) 

filepath="weights-improvement-VGG16-LOGO7-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

earlyStopping=EarlyStopping(monitor='val_acc', patience=4, verbose=0, mode='auto')
callbacks_list = [earlyStopping, checkpoint]
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy', 'categorical_crossentropy'])


h = model.fit(X_train, y_train, validation_data=(X_val, y_val),
          batch_size=16, nb_epoch=epochs, verbose=1,callbacks=callbacks_list)




/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py:16: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 2256 samples, validate on 564 samples
Epoch 1/100
2256/2256 [==============================] - 1300s - loss: 1.2554 - acc: 0.5559 - categorical_crossentropy: 1.2554 - val_loss: 1.0907 - val_acc: 0.5621 - val_categorical_crossentropy: 1.0907
Epoch 2/100
2256/2256 [==============================] - 1299s - loss: 0.9812 - acc: 0.6383 - categorical_crossentropy: 0.9812 - val_loss: 1.1574 - val_acc: 0.5940 - val_categorical_crossentropy: 1.1574
Epoch 3/100
2256/2256 [==============================] - 1298s - loss: 0.8655 - acc: 0.6658 - categorical_crossentropy: 0.8655 - val_loss: 1.0349 - val_acc: 0.6223 - val_categorical_crossentropy: 1.0349
Epoch 4/100
2256/2256 [==============================] - 1299s - loss: 0.7872 - acc: 0.6875 - categorical_crossentropy: 0.7872 - val_loss: 0.8636 - val_acc: 0.6454 - val_categorical_crossentropy: 0.8636
Epoch 5/100
2256/2256 [==============================] - 1299s - loss: 0.7365 - acc: 0.7083 - categorical_crossentropy: 0.7365 - val_loss: 0.

In [5]:
X_train.shape

(2256, 224, 224, 3)